In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras import backend as K
import tensorflow as tf
from category_encoders.binary import BinaryEncoder
import category_encoders as ce

Using TensorFlow backend.


In [2]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
K.set_session(sess)

In [4]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [5]:
# load dataset
dataframe = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [8]:
url = "unit-3/train_features.csv"
df = pd.read_csv(url, parse_dates=['date_recorded'],index_col='id' )

In [9]:
df['timestamp'] = df.date_recorded.apply(lambda d: d.timestamp()/ 10 ** 9)
df_ = df.drop(['date_recorded', 'region_code', 'district_code', 'region'], inplace = False, axis=1)
# df_ = df.drop(['date_recorded', 'district_code', 'region'], inplace = False, axis=1)
df_['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}' , axis=1)
# df_ = df_.apply(lambda x: x.astype(str).str.lower())
train_input_columns = list(df_.columns)
train_numeric_columns = df_.select_dtypes(exclude=['object']).columns
df_.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.300061,Iringa_5
8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.362528,Mara_2
34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1.361750,Manyara_4
67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.359331,Mtwara_63
19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.310515,Kagera_1


In [10]:
dc = 'status_group'

In [11]:
yurl = 'unit-3/train_labels.csv'
dfy = pd.read_csv(yurl, index_col='id' )
dfy.shape
y = dfy[dc]

In [12]:
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [13]:
oc = df_.select_dtypes(include=['object']).columns
oc

Index(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'lga', 'ward',
       'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name',
       'permit', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'region_district'],
      dtype='object')

In [14]:
hot = []
binary = []
for o in oc:
    if df_[o].unique().shape[0] > 127:
        print(df_[o].unique().shape[0], o)
        binary.append(o)
    else:
        hot.append(o)

1898 funder
2146 installer
37400 wpt_name
19288 subvillage
2092 ward
2697 scheme_name
132 region_district


In [15]:
test_url = "unit-3/test_features.csv"
df = pd.read_csv(test_url, parse_dates=['date_recorded'],index_col='id' )
df['timestamp'] = df.date_recorded.apply(lambda d: d.timestamp()/ 10 ** 9)
dft = df.drop(['date_recorded', 'region_code', 'district_code', 'region'], inplace = False, axis=1)
#dft = df.drop(['date_recorded', 'district_code', 'region'], inplace = False, axis=1)
dft['region_district'] = df.apply(lambda row: f'{row.region}_{row.district_code}' , axis=1)
# dft = dft.apply(lambda x: x.astype(str).str.lower())
test_input_columns = list(dft.columns)
# dft[dc] = ['fuctional'] * dft.shape[0]
dft.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,
50785,0.0,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,1.359936,Manyara_3
51630,0.0,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,...,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,1.359936,Arusha_2
17168,0.0,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,1.359677,Singida_2
45559,0.0,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,...,good,dry,dry,shallow well,shallow well,groundwater,other,other,1.358813,Lindi_43
49871,500.0,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.364342,Ruvuma_3


In [16]:
test__ = list(dft.columns)
for c in train_input_columns:
    if c not in test_input_columns:
        print(f'{c} not in test')
    else:
        test__.remove(c)
print(test__)
        

[]


In [17]:
encoders = Pipeline([
#                 ('vect', tfidf),
                ('binary', BinaryEncoder(cols=binary)),
                ('onehot', ce.OneHotEncoder(use_cat_names=True,cols=hot))
            ])
df_l = df_.shape[0]
both = pd.concat([df_,dft])
print(df_l)
both.head()

59400


,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,timestamp,region_district
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,...,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.300061,Iringa_5
8776,0.0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.362528,Mara_2
34310,25.0,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,...,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1.361750,Manyara_4
67743,0.0,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,...,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.359331,Mtwara_63
19728,0.0,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.310515,Kagera_1


In [18]:
both_  = encoders.fit_transform(both)

In [19]:
df__  = both_.iloc[0:df_l]
dft_ = both_.iloc[df_l:]
df_l, both_.shape, df__.shape, dft_.shape

(59400, (73758, 404), (59400, 404), (14358, 404))

In [20]:
def pump_baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=404, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [22]:
estimator = KerasClassifier(build_fn=pump_baseline_model, epochs=6000, batch_size=5, verbose=0)

In [23]:
# results = cross_val_score(estimator, df__, dummy_y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
model = pump_baseline_model()
model.fit(x=df__, y=dummy_y,epochs=6000)

Epoch 1/6000
59400/59400 [==============================] - 5s 78us/step - loss: 1.0037 - acc: 0.5868
Epoch 2/6000
59400/59400 [==============================] - 4s 72us/step - loss: 0.8070 - acc: 0.6156
Epoch 3/6000
59400/59400 [==============================] - 4s 73us/step - loss: 0.7394 - acc: 0.6981
Epoch 4/6000
59400/59400 [==============================] - 4s 74us/step - loss: 0.6947 - acc: 0.7235
Epoch 5/6000
59400/59400 [==============================] - 5s 76us/step - loss: 0.6760 - acc: 0.7322
Epoch 6/6000
59400/59400 [==============================] - 4s 75us/step - loss: 0.6660 - acc: 0.7355
Epoch 7/6000
59400/59400 [==============================] - 5s 76us/step - loss: 0.6610 - acc: 0.7385
Epoch 8/6000
59400/59400 [==============================] - 5s 77us/step - loss: 0.6528 - acc: 0.7415
Epoch 9/6000
59400/59400 [==============================] - 5s 76us/step - loss: 0.6463 - acc: 0.7419
Epoch 10/6000
59400/59400 [==============================] - 5s 78us/step - loss: 

59400/59400 [==============================] - 4s 64us/step - loss: 0.6000 - acc: 0.7611
Epoch 81/6000
59400/59400 [==============================] - 3s 58us/step - loss: 0.6015 - acc: 0.7604
Epoch 82/6000
59400/59400 [==============================] - 3s 51us/step - loss: 0.6035 - acc: 0.7594
Epoch 83/6000
59400/59400 [==============================] - 4s 60us/step - loss: 0.5992 - acc: 0.7609
Epoch 84/6000
59400/59400 [==============================] - 4s 62us/step - loss: 0.6000 - acc: 0.7602
Epoch 85/6000
59400/59400 [==============================] - 5s 86us/step - loss: 0.6006 - acc: 0.7604
Epoch 86/6000
59400/59400 [==============================] - 4s 67us/step - loss: 0.5988 - acc: 0.7608
Epoch 87/6000
59400/59400 [==============================] - 3s 58us/step - loss: 0.6001 - acc: 0.7593
Epoch 88/6000
59400/59400 [==============================] - 3s 54us/step - loss: 0.6040 - acc: 0.7594
Epoch 89/6000
59400/59400 [==============================] - 4s 71us/step - loss: 0.597

59400/59400 [==============================] - 4s 70us/step - loss: 0.5946 - acc: 0.7646
Epoch 160/6000
59400/59400 [==============================] - 4s 62us/step - loss: 0.5939 - acc: 0.7635
Epoch 161/6000
59400/59400 [==============================] - 4s 62us/step - loss: 0.5935 - acc: 0.7623
Epoch 162/6000
59400/59400 [==============================] - 4s 63us/step - loss: 0.5969 - acc: 0.7623
Epoch 163/6000
59400/59400 [==============================] - 4s 60us/step - loss: 0.5959 - acc: 0.7618
Epoch 164/6000
59400/59400 [==============================] - 4s 68us/step - loss: 0.5949 - acc: 0.7633
Epoch 165/6000
59400/59400 [==============================] - 4s 59us/step - loss: 0.5955 - acc: 0.7625
Epoch 166/6000
59400/59400 [==============================] - 4s 62us/step - loss: 0.5953 - acc: 0.7624
Epoch 167/6000
59400/59400 [==============================] - 4s 59us/step - loss: 0.5941 - acc: 0.7634
Epoch 168/6000
59400/59400 [==============================] - 4s 62us/step - lo

59400/59400 [==============================] - 4s 65us/step - loss: 0.5934 - acc: 0.7654
Epoch 238/6000
59400/59400 [==============================] - 4s 60us/step - loss: 0.5950 - acc: 0.7641
Epoch 239/6000
59400/59400 [==============================] - 4s 60us/step - loss: 0.5908 - acc: 0.7643
Epoch 240/6000
59400/59400 [==============================] - 3s 52us/step - loss: 0.5922 - acc: 0.7643
Epoch 241/6000
59400/59400 [==============================] - 4s 61us/step - loss: 0.5924 - acc: 0.7645
Epoch 242/6000
59400/59400 [==============================] - 3s 53us/step - loss: 0.5924 - acc: 0.7645
Epoch 243/6000
59400/59400 [==============================] - 3s 52us/step - loss: 0.5932 - acc: 0.7636
Epoch 244/6000
59400/59400 [==============================] - 3s 53us/step - loss: 0.5919 - acc: 0.7638
Epoch 245/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5920 - acc: 0.7644
Epoch 246/6000
59400/59400 [==============================] - 3s 50us/step - lo

59400/59400 [==============================] - 3s 50us/step - loss: 0.5910 - acc: 0.7654
Epoch 316/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5935 - acc: 0.7657
Epoch 317/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5950 - acc: 0.7642
Epoch 318/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5935 - acc: 0.7657
Epoch 319/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5917 - acc: 0.7646
Epoch 320/6000
59400/59400 [==============================] - 3s 51us/step - loss: 0.5922 - acc: 0.7646
Epoch 321/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5926 - acc: 0.7642
Epoch 322/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5919 - acc: 0.7655
Epoch 323/6000
59400/59400 [==============================] - 3s 50us/step - loss: 0.5903 - acc: 0.7660
Epoch 324/6000
59400/59400 [==============================] - 3s 50us/step - lo

In [127]:
p = model.predict(dft_, batch_size=None, verbose=0, steps=None)
p

array([[0.22859693, 0.03930179, 0.73210126],
       [0.5831133 , 0.04652163, 0.37036508],
       [0.46959814, 0.21885042, 0.31155148],
       ...,
       [0.92457575, 0.0597909 , 0.01563349],
       [0.9147806 , 0.03263792, 0.05258147],
       [0.00662169, 0.00510542, 0.98827285]], dtype=float32)

In [128]:
p.shape

(14358, 3)

In [133]:
with open('testxgb.keras.csv', 'w') as f:
    f.write('id,status_group\n')
    for fu,i in zip(p, dft_.index):
        index = np.argmax(fu)
        d = 'non functional' if index == 2 else ('functional' if index == 0 else 'functional needs repair')
        f.write(f"{i},{d}")
        f.write('\n')

Your submission scored 0.75915